In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

from odesolver.gen_ode_system import generate_ode_system

In [ ]:
generate_ode_system(output_dir="./../../flow_equations/", project_name="Nonlinear System", flow_equation_path="./../../notebooks/nonlinear_system/")

In [ ]:
!{sys.executable} -m pip install --use-feature=in-tree-build ./../../flow_equations/NonlinearSystem

### Evaluating the flow equation at given coordinates

In [ ]:
from nonlinearsystem.nonlinear_system import NonlinearSystem
ode_system = NonlinearSystem()
print(ode_system.dim)
print(ode_system.model)
print(ode_system.flow_variable)
print(ode_system.flow_parameters)

In [ ]:
from odesolver.coordinates import Coordinates
coordinates = Coordinates.from_data(np.random.randn(10, 2))
print("Dim", coordinates.dim())
print("Size", coordinates.size())
print("Shape", coordinates.shape)
print("Coordinates", coordinates)

In [ ]:
print("Flow", ode_system.flow(coordinates))

### Fixed Point Search

In [ ]:
from odesolver.recursive_search import RecursiveSearch
from odesolver.fixed_point_criterion import FixedPointCriterion
from odesolver.kmeans_clustering import KMeansClustering

In [ ]:
fixed_point_criterion = FixedPointCriterion()

recursive_fixed_point_search = RecursiveSearch(
    maximum_recursion_depth=18,
    n_branches_per_depth=[[100, 100]] + [[2, 2]] * 17,
    variable_ranges=[[-10.0, 10.0], [-10.0, 10.0]],
    criterion=fixed_point_criterion,
    flow_equations=ode_system,
    number_of_cubes_per_gpu_call=2000,
    maximum_number_of_gpu_calls=1000
)
recursive_fixed_point_search.eval("dynamic")

#### Solutions

In [ ]:
fixed_points = recursive_fixed_point_search.solutions()
print(fixed_points.transpose())

#### Cube index path of solutions

In [ ]:
fixed_point_cube_indices = recursive_fixed_point_search.solutions("cube_indices")
for fixed_point_cube_index in fixed_point_cube_indices:
    print(fixed_point_cube_index.depth, fixed_point_cube_index.indices)

#### KMeans clustering of solutions

In [ ]:
kmeans_clustering = KMeansClustering(
    maximum_expected_number_of_clusters=10,
    upper_bound_for_min_distance=0.0005,
    maximum_number_of_iterations=10
)

fixed_points = kmeans_clustering.eval(fixed_points)

In [ ]:
print(fixed_points)

### Jacobians

In [ ]:
jacobian_elements = ode_system.jacobians(fixed_points)
print("Jacobians\n", np.array(jacobian_elements.transpose().data).reshape(2, 2, 2))

In [ ]:
from odesolver.jacobians import Jacobians
jacobians = Jacobians.from_coor(elements=jacobian_elements)
jacobians.eval()

In [ ]:
for idx in range(fixed_points.size()):
    print("Jacobian\n", jacobians.jacobian(idx))
    print("Eigenvectors\n", jacobians.eigenvectors(idx))
    print("Eigenvalues\n", jacobians.eigenvalues(idx))

### Visualization

#### Mesh

In [ ]:
from odesolver.mesh import Mesh
mesh = Mesh(n_branches=[26, 31], variable_ranges=[[-3.5, 3.5], [-2.0, 3.0]])
Y, X = mesh.mgrid()
YY, XX = np.mgrid[-2.0:3.0:31j, -3.5:3.5:26j]
assert np.all(np.isclose(Y, YY)) and np.all(np.isclose(X, XX))

In [ ]:
vertices = mesh.vertices()
flow = ode_system.flow(vertices)
V, U = mesh.mgrid(data=flow)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.streamplot(X, Y, U, V, density=[2.0, 2.0], linewidth=1.0, arrowsize=0.7)
ax.scatter(fixed_points.data[0, :], fixed_points.data[1, :], s=80, c="black", marker="o");

### Evolution

In [ ]:
from odesolver.evolution import Evolution
from odevisualizationlib.evolution.stepper import RungaKutta4

#### Sample around fixed points

In [ ]:
def sample_around_fixed_points(fixed_points):
    samples = Coordinates.generate(dim=2, N=10)
    samples[:, :2] = fixed_points.data[:, :2]
    samples[:, 2:4] = fixed_points.data[:, :2] + 0.1
    samples[:, 4:6] = fixed_points.data[:, :2] - 0.1
    samples[0, 6:8] = fixed_points.data[0, :2] + 0.1
    samples[1, 6:8] = fixed_points.data[1, :2] - 0.1
    samples[0, 8:10] = fixed_points.data[0, :2] - 0.1
    samples[1, 8:10] = fixed_points.data[1, :2] + 0.1
    return samples

In [ ]:
samples = sample_around_fixed_points(fixed_points=fixed_points)

fig, axes = plt.subplots(ncols= 2, figsize=(12, 6))
axes[0].streamplot(X, Y, U, V, density=[1.6, 1.6], linewidth=1.0, arrowsize=0.7)
axes[0].scatter(fixed_points.data[0, :], fixed_points.data[1, :], s=80, c="black", marker="o")
axes[0].scatter(samples[0, :], samples[1, :], s=80, c="red", marker="x")

evolution = Evolution(flow_equations=ode_system)
stepper = RungaKutta4()
evolution.evolve(coordinates=samples, stepper=stepper, start_t=0.0, dt=0.01, n=300)

axes[1].streamplot(X, Y, U, V, density=[1.6, 1.6], linewidth=1.0, arrowsize=0.7)
axes[1].scatter(fixed_points.data[0, :], fixed_points.data[1, :], s=80, c="black", marker="o")
axes[1].scatter(samples[0, :], samples[1, :], s=80, c="red", marker="x");

In [ ]:
samples = sample_around_fixed_points(fixed_points=fixed_points)

fig, axes = plt.subplots(ncols= 2, figsize=(12, 6))
axes[0].streamplot(X, Y, U, V, density=[1.6, 1.6], linewidth=1.0, arrowsize=0.7)
axes[0].scatter(fixed_points.data[0, :], fixed_points.data[1, :], s=80, c="black", marker="o")
axes[0].scatter(samples[0, :], samples[1, :], s=80, c="red", marker="x")

evolution = Evolution(flow_equations=ode_system)
stepper = RungaKutta4()
evolution.evolve(coordinates=samples, stepper=stepper, start_t=0.0, dt=-0.002, n=420)

axes[1].streamplot(X, Y, U, V, density=[1.6, 1.6], linewidth=1.0, arrowsize=0.7)
axes[1].scatter(fixed_points.data[0, :], fixed_points.data[1, :], s=80, c="black", marker="o")
axes[1].scatter(samples[0, :], samples[1, :], s=80, c="red", marker="x");

### Separatrizes

In [ ]:
from odevisualizationlib.evolution.observer import EvolutionObserver, DivergentFlow, NoChange, OutOfRangeCondition, Intersection
divergent_flow = DivergentFlow(ode_system._flow, 1e6)
no_change = NoChange([1e-6, 1e-6])
out_of_range_condition = OutOfRangeCondition([[-3.5, 3.5], [-2.0, 3.0]])
intersection = Intersection([0.1, 0.1], [], [], True)

evolution_observer = EvolutionObserver([divergent_flow, no_change, out_of_range_condition, intersection])

In [ ]:
from odevisualizationlib.modes import Evolution as VFEvolution
vf_evolution = VFEvolution(ode_system._flow)

In [ ]:
from odevisualizationlib.modes import Separatrizes
separatrizes = Separatrizes(10, [0.1, 0.1], 470, ode_system._flow, ode_system._jacobians)
stepper = RungaKutta4()

In [ ]:
separatrizes.eval(fixed_points._devdat, 0.002, vf_evolution, stepper, intersection);

In [ ]:
detected_intersections = np.array(intersection.detected_intersections())
# detected_intersections = detected_intersections[np.all([~np.isnan(detected_intersections[:, 0]), ~np.isnan(detected_intersections[:, 1])], axis=0)]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.streamplot(X, Y, U, V, density=[2.0, 2.0], linewidth=1.0, arrowsize=0.7)
ax.scatter(fixed_points.data[0, :], fixed_points.data[1, :], s=80, c="black", marker="o")
ax.scatter(detected_intersections[:, 0], detected_intersections[:, 1], s=80, c="red", marker="x")

#### Evolution Observer

In [ ]:
from odevisualizationlib.evolution.observer import EvolutionObserver, DivergentFlow, NoChange, OutOfRangeCondition, Intersection
divergent_flow = DivergentFlow(ode_system._flow, 1e6)
no_change = NoChange([1e-6, 1e-6])
out_of_range_condition = OutOfRangeCondition([[-3.5, 3.5], [-2.0, 3.0]])

evolution_observer = EvolutionObserver([divergent_flow, no_change, out_of_range_condition])

In [ ]:
coordinates = Coordinates.from_data(np.random.randn(100, 2) * 0.2)
coordinates[1, :] = coordinates[1, :] + 1.0
fig, ax = plt.subplots(figsize=(8, 8))
ax.streamplot(X, Y, U, V, density=[2.0, 2.0], linewidth=1.0, arrowsize=0.7)
ax.scatter(coordinates.data[0, :], coordinates.data[1, :], s=80, c="black", marker="o");

In [ ]:
evolution = Evolution(flow_equations=ode_system)
stepper = RungaKutta4()
evolution.evolve(coordinates=coordinates, stepper=stepper, start_t=0.0, dt=-0.01, n=300, observer=evolution_observer, observe_every_ith_time_step=1)

In [ ]:
print(fixed_points)